# analyze video(s)
Various cells for analyzing videos, groups of videos, etc...

In [2]:
# inits
import subprocess
import glob
import yaml
import sys
import os
root_dir = r'D:\github\fish-tracking'  # github repo
sys.path.append(os.path.abspath(root_dir))  # add partent directory
%load_ext autoreload
%autoreload 2

# load config
with open(os.path.join(root_dir, 'config.yaml'), 'r') as file:
    cfg = yaml.safe_load(file)


def analyze_vid(video, model_name, skeleton,
             batch_size=16, make_tracking_vid=False, overwrite=False):
    """ wrapper for analyze_video.py """
    
    cmd = 'python analyze_video.py {} {} {} -batch_size {}'.format(
        video, model_name, skeleton, batch_size)
    
    if overwrite:
        cmd += ' -overwrite'
    if make_tracking_vid:
        cmd += ' -makevideo'
    
    subprocess.run(cmd)

    
def get_fixed_vids(overwrite=False):
    """
    get all vids to be analyzed for head fixed stimulation videos
    todo: accept list of folders and only return those vids
    """
    
    vids_to_analyze = []
    vid_folders = [f.path for f in os.scandir(cfg['fixed']['data_dir']) if f.is_dir()]

    # find all videos
    for vid_folder in vid_folders:
        vids = glob.glob(os.path.join(vid_folder, '*.avi'))
        vids = [vid for vid in vids if 'concatenated' not in vid and 'tracking' not in vid]    # don't analyze vids that are concatenated, or tracking videos
        vids_to_analyze += vids
    
    # remove already-tracked vids
    if not overwrite:
        vids_to_analyze = [v for v in vids_to_analyze if not os.path.exists(os.path.splitext(v)[0] + '_tracking.csv')]
    
    print('found {} videos'.format(len(vids_to_analyze)))
            
    return vids_to_analyze


def get_free_vids(overwrite=False):
    """
    get all vids to be analyzed for head free tank videos
    only looks for concatenated.avi vids
    todo: accept list of folders and only return those vids
    """
    
    vids_to_analyze = []
    vid_folders = [f.path for f in os.scandir(cfg['free']['data_dir']) if f.is_dir()]

    # find all videos
    for vid_folder in vid_folders:
        vid = os.path.join(vid_folder, 'concatenated.avi')
        if os.path.exists(vid):
            vids_to_analyze.append(vid)
        else:
            print(f'{vid}: File does not exist (must run concatenation scripts first)!')
    
    # remove already-tracked vids
    if not overwrite:
        vids_to_analyze = [v for v in vids_to_analyze if not os.path.exists(os.path.splitext(v)[0] + '_tracking.csv')]
    
    print('found {} videos'.format(len(vids_to_analyze)))
            
    return vids_to_analyze
    

## analyze single video
analyze a single video, and make video with tracking overlaid. must specify the model and skeleton.

In [87]:
# analyze single video

# settings
video = r'Z:\C1\free\vids\20201105_lazy\concatenated.avi'
# model_name = r'D:\github\fish-tracking\train\models\tank_model.h5'
# skeleton = r'D:\github\fish-tracking\label\skeletons\tank_skeleton_3.csv'
model_name = r'D:\github\fish-tracking\train\models\stim_model.h5'
skeleton = r'D:\github\fish-tracking\label\skeletons\stim_skeleton.csv'
overwrite = True
make_tracking_vid = True
batch_size = 16

cmd = analyze_vid(video, model_name, skeleton,
                  batch_size=batch_size, make_tracking_vid=make_tracking_vid, overwrite=overwrite)

## analyze all folders for project
given project name (`free` or `fixed`) analyzes all videos that have not yet been analyzed. edit `config.yaml` to define folders, models, skeletons...

In [ ]:
# settings
vid_type = 'free'    # 'free' or 'fixed'
overwrite = False    # whether to overwrite existing tracking
batch_size = 32
make_tracking_vid = True


if vid_type=='free':
    vids = get_free_vids(overwrite)
elif vid_type=='fixed':
    vids = get_fixed_vids(overwrite)

for vid in vids:
    print(f'analyzing video: {vid}')
    analyze_vid(vid, cfg[vid_type]['model'], cfg[vid_type]['skeleton'], 
                batch_size=batch_size, make_tracking_vid=make_tracking_vid)


Z:\C1\free\vids\20201020_Greg\concatenated.avi: File does not exist (must run concatenation scripts first)!
found 50 videos
analyzing video: Z:\C1\free\vids\20201126_Joao\concatenated.avi
analyzing video: Z:\C1\free\vids\20201128_Kyle\concatenated.avi
analyzing video: Z:\C1\free\vids\20201128_Lazy\concatenated.avi
